In [8]:
import json
import pandas as pd
import re
## Training set preperation
dict_mapping = {'Presenting Irrelevant Data (Red Herring)': 0,
 'Bandwagon': 1,
 'Smears': 2,
 'Glittering generalities (Virtue)': 3,
 'Causal Oversimplification': 4,
 'Whataboutism': 5,
 'Loaded Language': 6,
 'Exaggeration/Minimisation': 7,
 'Repetition': 8,
 'Thought-terminating cliché': 9,
 'Name calling/Labeling': 10,
 'Appeal to authority': 11,
 'Black-and-white Fallacy/Dictatorship': 12,
 'Obfuscation, Intentional vagueness, Confusion': 13,
 'Reductio ad hitlerum': 14,
 'Appeal to fear/prejudice': 15,
 "Misrepresentation of Someone's Position (Straw Man)": 16,
 'Flag-waving': 17,
 'Slogans': 18,
 'Doubt': 19,
 'Bandwagon_1': 20,
 'Whataboutism_1': 21,
 'Appeal to fear/prejudice_1': 22,
 'Flag-waving_1': 23,
 'Appeal to authority_1': 24,
}


def dataset_f(data,dict_mapping):
    set_x = set()
    for element in data:
        for label in element['labels']:
            set_x.add(label)
    
    data_id = []
    for element in data:
        dict_x = {}
        dict_x["text"] = element["text"]
        dict_x["labels"] = element["labels"]
        dict_x["id"] = element["id"]
        if len(element['labels']):
            data_id.append(dict_x)
    
    unprocessed = []
    processed = []
    labels = []
    ID = []
    for element in data_id:
        cleaned_text = re.sub(r'&', 'and',element['text'])
        pattern = r'\\n'
        cleaned_text = re.sub(pattern, ' ',cleaned_text)
        pattern = r'[^a-zA-Z0-9\s]+'
        cleaned_text = re.sub(pattern, '', cleaned_text).lower()
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        if(cleaned_text != ''):
            unprocessed.append(element['text'])
            processed.append(cleaned_text)
            labels.append(element['labels'])
            ID.append(element['id'])
        
    data = {'text': unprocessed, 'aug_text': processed, 'label': labels,'id':ID}
    df = pd.DataFrame(data)

    for index,row in df.iterrows():
        l_labels = row['label']
        empty_list = []
        for label in l_labels:
            empty_list.append(dict_mapping[label])
        row['label'] = empty_list

    return df
    
with open(r'data_given_to_work_with/subtask1/train.json', 'rb') as json_file:
    data = json.load(json_file)

df_train = dataset_f(data,dict_mapping)

with open(r'data_given_to_work_with/subtask1/validation.json', 'rb') as json_file:
    data = json.load(json_file)

df_val = dataset_f(data,dict_mapping)


df_train_full = pd.concat([df_train,df_val],axis=0)
df_exploded_train = df_train_full.explode('label')

condition = df_exploded_train['label'].isin([1, 5, 15, 17, 11])
df_duplicated = df_exploded_train[condition].copy()
mapping_dict = {1: 20, 5: 21, 15: 22, 17: 23, 11: 24}
df_duplicated['label'] = df_duplicated['label'].map(mapping_dict)
df_duplicated = df_duplicated.drop_duplicates()
result_df = pd.concat([df_exploded_train, df_duplicated], ignore_index=True)
result_df.to_csv('Merged_train_for_HypEmo.csv')

In [11]:
def dataset_f(data,dict_mapping):
    data_id = []
    for element in data:
        dict_x = {}
        dict_x["text"] = element["text"]
        dict_x["id"] = element["id"]
        data_id.append(dict_x)
        
    unprocessed = []
    processed = []
    labels = []
    ID = []
    
    print(len(data_id))
    for element in data_id:
        unprocessed.append(element['text'])
        cleaned_text = re.sub(r'&', 'and',element['text'])
        cleaned_text = cleaned_text.lower()
        processed.append(cleaned_text)
        x = 9
        labels.append(x)
        ID.append(element['id'])
    
    data = {'text': unprocessed, 'aug_text': processed, 'label': labels,'id':ID}
    df = pd.DataFrame(data)
    return df

with open(r'data_given_to_work_with/subtask1/en_subtask1_test_unlabeled.json', 'rb') as json_file:
    data = json.load(json_file)
df_test_eng_full = dataset_f(data,dict_mapping)


1000
1500


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open('Files_for_task1a_input/ar_subtask1_test_unlabeled.json', 'rb') as json_file:
    data = json.load(json_file)
df_test_arab =  dataset_f(data,dict_mapping)
src_text = list(df_test_arab["text"])
model_name = "Helsinki-NLP/opus-mt-ar-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)
t_text = []
batch_size = 16
num_samples = len(src_text)
translations = []
for i in tqdm(range(0, num_samples, batch_size)):
    batch_src_text = src_text[i:i + batch_size]
    batch_src_text = tokenizer(batch_src_text, return_tensors="pt", padding=True, truncation=True).to(device)
    batch_translated = model.generate(**batch_src_text).to("cpu")
    translations.extend(batch_translated)
for t in tqdm(translations):
    t_text.append(tokenizer.decode(t, skip_special_tokens=True))    
df_test_arab["aug_text"] = t_text


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with open('Files_for_task1a_input/bg_subtask1_test_unlabeled.json', 'rb') as json_file:
    data = json.load(json_file)
df_test_bulg =  dataset_f(data,dict_mapping)
src_text = list(df_test_bulg["text"])
model_name = "Helsinki-NLP/opus-mt-bg-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)
t_text = []
batch_size = 16
num_samples = len(src_text)
translations = []
for i in tqdm(range(0, num_samples, batch_size)):
    batch_src_text = src_text[i:i + batch_size]
    batch_src_text = tokenizer(batch_src_text, return_tensors="pt", padding=True, truncation=True).to(device)
    batch_translated = model.generate(**batch_src_text).to("cpu")
    translations.extend(batch_translated)
for t in tqdm(translations):
    t_text.append(tokenizer.decode(t, skip_special_tokens=True).lower())    
df_test_bulg["aug_text"] = t_text

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with open('Files_for_task1a_input/mk_subtask1_test_unlabeled.json', 'rb') as json_file:
    data = json.load(json_file)
    
df_test_mace =  dataset_f(data,dict_mapping)
src_text = list(df_test_mace["text"])
model_name = "Helsinki-NLP/opus-mt-mk-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

t_text = []
batch_size = 16
num_samples = len(src_text)
translations = []

for i in tqdm(range(0, num_samples, batch_size)):
    batch_src_text = src_text[i:i + batch_size]
    batch_src_text = tokenizer(batch_src_text, return_tensors="pt", padding=True, truncation=True).to(device)
    batch_translated = model.generate(**batch_src_text).to("cpu")
    translations.extend(batch_translated)

for t in tqdm(translations):
    t_text.append(tokenizer.decode(t, skip_special_tokens=True).lower())

df_test_mace["aug_text"] = t_text

In [24]:
df_test_full = pd.concat([df_dev_eng_full,df_test_eng_full,df_test_arab,df_test_bulg,df_test_mace],ignore_index=True)
import pandas as pd
df_test_full = pd.read_csv('Merged_test_for_HypEmo.csv')